In [1]:
import os
import glob
import keras

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from skimage.transform import resize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from tensorflow.examples.tutorials.mnist import input_data

/Users/derekzhao/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def resize_data(X):
  image_list = list()
  for image in X:
    image_list.append(resize(image, output_shape=(32, 32)))
  X = np.array(image_list)
  # print(X.shape)
  # print(len(image_list))
  assert(X.shape == (len(image_list), 32, 32, 1))
  return X

In [3]:
num_final_hidden_layer = 32
num_classes = 10
batch_size = 128

checkpoint_dir = '../experiments/baseline-mnist'
if not os.path.isdir(checkpoint_dir):
    os.makedirs(checkpoint_dir)
    
datagen = ImageDataGenerator()

mnist = input_data.read_data_sets('../data/MNIST/', reshape=False)
X_train = mnist.train.images
X_train = resize_data(X_train)
y_train = mnist.train.labels

X_val = mnist.validation.images
X_val = resize_data(X_val)
y_val = mnist.validation.labels

X_test = mnist.test.images
X_test = resize_data(X_test)
y_test = mnist.test.labels

train_datagen = datagen.flow(X_train, y_train)
validation_datagen = datagen.flow(X_val, y_val)
test_datagen = datagen.flow(X_test, y_test)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../data/MNIST/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


/Users/derekzhao/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [4]:
model = Sequential()
model.add(Conv2D(32, kernel_size=4, strides=2, activation='relu',
                 input_shape=(32, 32, 1), padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.1, epsilon=1e-5))
model.add(Conv2D(64, kernel_size=4, strides=2, activation='relu', padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.1, epsilon=1e-5))
model.add(Conv2D(128, kernel_size=4, strides=2, activation='relu', padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.1, epsilon=1e-5))
model.add(Conv2D(128, kernel_size=4, strides=2, activation='relu', padding='same'))
model.add(BatchNormalization(axis=-1, momentum=0.1, epsilon=1e-5))
model.add(Conv2D(num_final_hidden_layer, kernel_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

In [5]:
stopping = EarlyStopping(monitor='val_acc', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint(
    os.path.join(checkpoint_dir, 'checkpoint'),
    monitor='val_acc', save_best_only=True)
model.compile(optimizer=Adam(lr=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
history = model.fit_generator(
    train_datagen, validation_data=validation_datagen,
    callbacks=[stopping, checkpoint], epochs=100,
    steps_per_epoch=len(train_datagen),
    validation_steps=len(validation_datagen))

Epoch 1/100
1719/1719 [==============================] - 58s 33ms/step - loss: 0.6010 - acc: 0.8137 - val_loss: 0.2447 - val_acc: 0.9282
Epoch 2/100
1719/1719 [==============================] - 59s 34ms/step - loss: 0.1884 - acc: 0.9447 - val_loss: 0.1501 - val_acc: 0.9564
Epoch 3/100
1719/1719 [==============================] - 63s 37ms/step - loss: 0.1181 - acc: 0.9663 - val_loss: 0.1123 - val_acc: 0.9656
Epoch 4/100
 723/1719 [===========>..................] - ETA: 34s - loss: 0.0878 - acc: 0.9758

KeyboardInterrupt: 

In [ ]:
result = model.evaluate_generator(test_datagen, steps=len(test_datagen))
result